In [1]:
!pip install scikit-fuzzy

In [2]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

# ---------- Load Gene Expression Model ----------
gene_model = load_model('best_gene_model.h5')  # Ensure this matches gene model path

# ---------- Load Gene Data ----------
df_raw = pd.read_excel('C:\\Users\\Dell\\Downloads\\GSE7553.XLSX', sheet_name='Sheet1', index_col=0)
df = df_raw.transpose()
X = df.astype(float).values
y_raw = [label.split('.')[0] for label in df_raw.columns]

encoder = LabelEncoder()
y = encoder.fit_transform(y_raw)
y_encoded = to_categorical(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [3]:
# Load the trained model
image_model = load_model('best_testify_model.h5')  # Ensure this file is in your working directory or give full path

# Use the exact correct path to the training folder
train_dir = r"C:\Users\DELL\Downloads\Skin cancer ISIC The International Skin Imaging Collaboration\Train"

# Check if the path exists before using it
if not os.path.exists(train_dir):
    raise FileNotFoundError(f"Directory not found: {train_dir}")

# Get the list of class names (folder names inside Train)
class_names = sorted([
    d for d in os.listdir(train_dir)
    if os.path.isdir(os.path.join(train_dir, d))
])

print("Class names found:", class_names)

Class names found: ['actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']


In [4]:
def fuzzy_fuse(image_score, gene_score):
    def triangular_membership(x, a, b, c):
        if x <= a or x >= c:
            return 0
        elif a < x <= b:
            return (x - a) / (b - a)
        else:  # b < x < c
            return (c - x) / (c - b)

    def fuzzify_confidence(score):
        return {
            "low": triangular_membership(score, 0.0, 0.2, 0.4),
            "medium": triangular_membership(score, 0.3, 0.5, 0.7),
            "high": triangular_membership(score, 0.6, 0.8, 1.0)
        }

    def apply_rules(image_fuzzy, gene_fuzzy):
        return {
            "low": max(
                min(image_fuzzy["low"], gene_fuzzy["low"]),
                min(image_fuzzy["low"], gene_fuzzy["medium"]),
                min(image_fuzzy["medium"], gene_fuzzy["low"])
            ),
            "medium": max(
                min(image_fuzzy["medium"], gene_fuzzy["medium"]),
                min(image_fuzzy["high"], gene_fuzzy["low"]),
                min(image_fuzzy["low"], gene_fuzzy["high"])
            ),
            "high": max(
                min(image_fuzzy["high"], gene_fuzzy["high"]),
                min(image_fuzzy["medium"], gene_fuzzy["high"]),
                min(image_fuzzy["high"], gene_fuzzy["medium"])
            )
        }

    def defuzzify(rules):
        output_values = {"low": 0.3, "medium": 0.6, "high": 0.9}
        numerator = sum(rules[label] * output_values[label] for label in rules)
        denominator = sum(rules[label] for label in rules)
        return numerator / denominator if denominator != 0 else 0.0

    image_fuzzy = fuzzify_confidence(image_score)
    gene_fuzzy = fuzzify_confidence(gene_score)
    rules = apply_rules(image_fuzzy, gene_fuzzy)
    return defuzzify(rules)

In [5]:
# ---------- Predict and Fuse ----------
while True:
    try:
        gene_idx = input(f"Enter gene sample index (0 to {X_scaled.shape[0]-1}) or 'exit': ")
        if gene_idx.lower() == 'exit':
            break
        gene_idx = int(gene_idx)
        gene_sample = X_scaled[gene_idx:gene_idx+1]
        gene_pred = gene_model.predict(gene_sample, verbose=0)[0]
        gene_conf = np.max(gene_pred)
        gene_label = encoder.inverse_transform([np.argmax(gene_pred)])[0]

        img_path = input("Enter image path or 'exit': ")
        if img_path.lower() == 'exit':
            break
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        image_pred = image_model.predict(img_array, verbose=0)[0]
        image_conf = np.max(image_pred)
        print("Prediction output of image :", image_pred)
        print("Prediction output of gene :", gene_pred)
        print("Class names:", class_names)
        print("maximum from gene prediction :", np.max(gene_pred))
        print("maximum from image prediction :", np.max(image_pred))

        image_label = class_names[np.argmax(image_pred)]

        fused_conf = fuzzy_fuse(image_conf, gene_conf)

        print(f"\nGene Prediction: {gene_label} (Confidence: {gene_conf:.2f})")
        print(f"Image Prediction: {image_label} (Confidence: {image_conf:.2f})")
        print(f"Fused Confidence Score: {fused_conf:.2f}")

        if image_conf > gene_conf :
            final_decision = image_label
            trusted_model = "image"
        elif gene_conf > image_conf :
            final_decision = gene_label
            trusted_model = "Gene"
        else:
            final_decision = image_label if fused_conf >= 0.75 else gene_label


        if trusted_model == "image":
            print(f"Final Decision: {final_decision} (Image-Based)")
        elif trusted_model == "Gene":
            print(f"Final Decision: {final_decision} (Gene-Based)")
        else:
            print("Ambiguous prediction. Consider further validation.")

    except Exception as e:
        print(f"Error: {e}")
        continue

Enter gene sample index (0 to 86) or 'exit': 1
Enter image path or 'exit': C:\Users\DELL\Downloads\img\sqamous (1).jpg
Prediction output of image : [0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 1.9722471e-13
 5.6594748e-20 1.8246418e-29 0.0000000e+00 2.2877663e-22]
Prediction output of gene : [9.9899215e-01 5.8270717e-04 1.9329079e-08 1.8687494e-04 2.6739395e-05
 1.8480174e-04 2.6805346e-05]
Class names: ['actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']
maximum from gene prediction : 0.99899215
maximum from image prediction : 1.0

Gene Prediction: Basal cell carcinoma (Confidence: 1.00)
Image Prediction: melanoma (Confidence: 1.00)
Fused Confidence Score: 0.00
Final Decision: melanoma (Image-Based)
Enter gene sample index (0 to 86) or 'exit': exit


In [7]:
for class_name in class_names:
    class_path = os.path.join(train_dir, class_name)
    print(f"\nClass: {class_name}")
    print(os.listdir(class_path)[:5])  # Print first 5 filenames



Class: actinic keratosis
['ISIC_0025780.jpg', 'ISIC_0025803.jpg', 'ISIC_0025825.jpg', 'ISIC_0025953.jpg', 'ISIC_0025957.jpg']

Class: basal cell carcinoma
['ISIC_0024504.jpg', 'ISIC_0024515.jpg', 'ISIC_0024550.jpg', 'ISIC_0024564.jpg', 'ISIC_0024572.jpg']

Class: dermatofibroma
['ISIC_0025223.jpg', 'ISIC_0025302.jpg', 'ISIC_0025314.jpg', 'ISIC_0025373.jpg', 'ISIC_0025504.jpg']

Class: melanoma
['ISIC_0000139.jpg', 'ISIC_0000141.jpg', 'ISIC_0000142.jpg', 'ISIC_0000143.jpg', 'ISIC_0000144.jpg']

Class: nevus
['ISIC_0000019.jpg', 'ISIC_0000020.jpg', 'ISIC_0000021.jpg', 'ISIC_0000023.jpg', 'ISIC_0000024.jpg']

Class: pigmented benign keratosis
['ISIC_0024435.jpg', 'ISIC_0024445.jpg', 'ISIC_0024446.jpg', 'ISIC_0024453.jpg', 'ISIC_0024471.jpg']

Class: seborrheic keratosis
['ISIC_0010810.jpg', 'ISIC_0010812.jpg', 'ISIC_0010834.jpg', 'ISIC_0010852.jpg', 'ISIC_0010860.jpg']

Class: squamous cell carcinoma
['ISIC_0024710.jpg', 'ISIC_0024843.jpg', 'ISIC_0024923.jpg', 'ISIC_0024925.jpg', 'ISIC_0

In [29]:
import os

image_folder = "C:\\Users\\DELL\\Downloads\\Skin cancer ISIC The International Skin Imaging Collaboration\\Train"

available_images = os.listdir(image_folder)

# List first 10 .npy image files
print("Available image files:", available_images[:10])


Available image files: ['actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']


In [33]:
print("Column names in your Excel file:")
print(gene_data.columns.tolist())


Column names in your Excel file:
['!Sample_characteristics_ch1', 'Basal cell carcinoma', 'Basal cell carcinoma.1', 'Basal cell carcinoma.2', 'Basal cell carcinoma.3', 'Melanoma in situ', 'Melanoma in situ.1', 'Primary Melanoma', 'Primary Melanoma.1', 'Metastatic Melanoma', 'Metastatic Melanoma.1', 'Metastatic Melanoma.2', 'Metastatic Melanoma.3', 'Metastatic Melanoma.4', 'Metastatic Melanoma.5', 'Metastatic Melanoma.6', 'Metastatic Melanoma.7', 'Normal Skin', 'Primary Melanoma.2', 'Squamous cell carcinoma', 'Squamous cell carcinoma.1', 'Squamous cell carcinoma.2', 'Squamous cell carcinoma.3', 'Squamous cell carcinoma.4', 'Basal cell carcinoma.4', 'Basal cell carcinoma.5', 'Basal cell carcinoma.6', 'Basal cell carcinoma.7', 'Basal cell carcinoma.8', 'Basal cell carcinoma.9', 'Basal cell carcinoma.10', 'Basal cell carcinoma.11', 'Basal cell carcinoma.12', 'Basal cell carcinoma.13', 'Basal cell carcinoma.14', 'Metastatic Melanoma.8', 'Metastatic Melanoma.9', 'Metastatic Melanoma.10', 'Met

In [39]:
import pandas as pd
import numpy as np

# ---------------- Load Gene Data ---------------- #
gene_data = pd.read_excel("C:\\Users\\DELL\\Downloads\\GSE7553.XLSX", sheet_name="Sheet1")

# Step 1: Set the first column (gene names) as the index
gene_data = gene_data.set_index(gene_data.columns[0])

# Step 2: Transpose so that samples become rows
gene_data = gene_data.transpose()

# Step 3: Reset index to make sample names a column
gene_data = gene_data.reset_index()
gene_data = gene_data.rename(columns={'index': 'sample_name'})

# Step 4: Extract label (cancer type) from sample_name
gene_data['label'] = gene_data['sample_name'].apply(lambda x: x.split('.')[0].strip())

# Step 5: Strip any extra whitespace from all string fields
gene_data['sample_name'] = gene_data['sample_name'].astype(str).str.strip()
gene_data['label'] = gene_data['label'].astype(str).str.strip()
gene_data.columns = gene_data.columns.str.strip()

# Optional: Check the structure
print("✅ Gene data shape:", gene_data.shape)
print(gene_data[['sample_name', 'label']].head())

# Step 6: Extract features and labels (if needed for model)
gene_features = gene_data.drop(columns=['sample_name', 'label']).astype(float).values
gene_labels = gene_data['label'].values
sample_names = gene_data['sample_name'].values


✅ Gene data shape: (87, 54677)
!Sample_characteristics_ch1             sample_name                 label
0                              Basal cell carcinoma  Basal cell carcinoma
1                            Basal cell carcinoma.1  Basal cell carcinoma
2                            Basal cell carcinoma.2  Basal cell carcinoma
3                            Basal cell carcinoma.3  Basal cell carcinoma
4                                  Melanoma in situ      Melanoma in situ


In [45]:
import os
import pandas as pd

# Load gene data
gene_data = pd.read_excel(r"C:\Users\DELL\Downloads\GSE7553.XLSX")
gene_data.columns = gene_data.columns.str.strip()

# Rename appropriately
gene_data = gene_data.rename(columns={
    gene_data.columns[0]: "sample_name",
    gene_data.columns[1]: "label"
})

# Clean text
gene_data['sample_name'] = gene_data['sample_name'].astype(str).str.strip().str.lower()
gene_data['label'] = gene_data['label'].astype(str).str.strip().str.lower()

print("✅ Gene data loaded:", gene_data.shape)

# Load image paths
image_folder = r"C:\Users\DELL\Downloads\Skin cancer ISIC The International Skin Imaging Collaboration\Train"
image_records = []

for label_folder in os.listdir(image_folder):
    folder_path = os.path.join(image_folder, label_folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.lower().endswith('.jpg'):
                image_name = os.path.splitext(file)[0].strip().lower()
                image_records.append({
                    'image_name': image_name,
                    'label': label_folder.strip().lower(),
                    'image_path': os.path.join(folder_path, file)
                })

image_df = pd.DataFrame(image_records)
print("✅ Image data collected:", image_df.shape)

# Merge by label
merged_df = pd.merge(gene_data, image_df, on="label", how="left")

# One image per gene sample
final_df = merged_df.groupby("sample_name").apply(lambda x: x.sample(1, random_state=42)).reset_index(drop=True)

print("✅ Final matched shape:", final_df.shape)
print(final_df[['sample_name', 'label', 'image_name']].head())

# Save if needed
# final_df.to_csv("gene_image_matched.csv", index=False)


✅ Gene data loaded: (54675, 88)
✅ Image data collected: (2239, 3)
✅ Final matched shape: (54675, 90)
  sample_name    label image_name
0   1007_s_at  3386.88        NaN
1     1053_at  1053.86        NaN
2      117_at  417.216        NaN
3      121_at  854.209        NaN
4   1255_g_at  61.7639        NaN


In [35]:
import os

# Path to your image folder
image_folder = r"C:\Users\DELL\Downloads\Skin cancer ISIC The International Skin Imaging Collaboration\Train"

# List all image files
image_files = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]

# Strip '.jpg' to get base names for matching
image_names = [os.path.splitext(f)[0].strip() for f in image_files]

# Filter gene data for only samples that have matching image files
matched_gene_data = gene_data[gene_data['sample_name'].isin(image_names)]

# Get list of matched image paths in the same order
matched_images = [
    os.path.join(image_folder, f"{name}.jpg") for name in matched_gene_data['sample_name']
]

print(f"✅ Matched samples: {len(matched_gene_data)} / {len(gene_data)}")


✅ Matched samples: 0 / 87


In [36]:
# Check sample names from gene data
print("Sample names from gene data:")
print(gene_data['sample_name'].head(10).tolist())

# Check image base names (without .jpg)
print("\nImage base filenames from folder:")
print(image_names[:10])


Sample names from gene data:
['Basal cell carcinoma', 'Basal cell carcinoma.1', 'Basal cell carcinoma.2', 'Basal cell carcinoma.3', 'Melanoma in situ', 'Melanoma in situ.1', 'Primary Melanoma', 'Primary Melanoma.1', 'Metastatic Melanoma', 'Metastatic Melanoma.1']

Image base filenames from folder:
[]


In [37]:
import os
import pandas as pd

# 1. Get all images with folder names as labels
image_folder = r"C:\Users\DELL\Downloads\Skin cancer ISIC The International Skin Imaging Collaboration\Train"

image_records = []
for label_folder in os.listdir(image_folder):
    full_folder_path = os.path.join(image_folder, label_folder)
    if os.path.isdir(full_folder_path):
        for file in os.listdir(full_folder_path):
            if file.lower().endswith(".jpg"):
                image_name = os.path.splitext(file)[0].strip()
                image_records.append({
                    'image_name': image_name,
                    'label': label_folder.strip().lower()
                })

image_df = pd.DataFrame(image_records)
print("✅ Image data collected:", image_df.shape)

# 2. Clean gene data column names
gene_data.columns = gene_data.columns.str.strip()
gene_data = gene_data.rename(columns={gene_data.columns[0]: "sample_name"})  # Rename the first column
gene_data['sample_name'] = gene_data['sample_name'].astype(str).str.strip().str.lower()
gene_data['label'] = gene_data['label'].astype(str).str.strip().str.lower()

# 3. Merge on label (if sample_name is not the image name)
merged = pd.merge(gene_data, image_df, on="label", how="inner")
print("✅ Merged data shape:", merged.shape)


✅ Image data collected: (2239, 2)
✅ Merged data shape: (7631, 54678)


In [30]:
print(gene_samples[:10])  # should print sample IDs like 'GSM176515'


['Basal cell carcinoma', 'Basal cell carcinoma.1', 'Basal cell carcinoma.2', 'Basal cell carcinoma.3', 'Melanoma in situ', 'Melanoma in situ.1', 'Primary Melanoma', 'Primary Melanoma.1', 'Metastatic Melanoma', 'Metastatic Melanoma.1']


In [23]:
print(gene_data.columns.tolist())

['!Sample_characteristics_ch1', 'Basal cell carcinoma', 'Basal cell carcinoma.1', 'Basal cell carcinoma.2', 'Basal cell carcinoma.3', 'Melanoma in situ', 'Melanoma in situ.1', 'Primary Melanoma', 'Primary Melanoma.1', 'Metastatic Melanoma', 'Metastatic Melanoma.1', 'Metastatic Melanoma.2', 'Metastatic Melanoma.3', 'Metastatic Melanoma.4', 'Metastatic Melanoma.5', 'Metastatic Melanoma.6', 'Metastatic Melanoma.7', 'Normal Skin', 'Primary Melanoma.2', 'Squamous cell carcinoma', 'Squamous cell carcinoma.1', 'Squamous cell carcinoma.2', 'Squamous cell carcinoma.3', 'Squamous cell carcinoma.4', 'Basal cell carcinoma.4', 'Basal cell carcinoma.5', 'Basal cell carcinoma.6', 'Basal cell carcinoma.7', 'Basal cell carcinoma.8', 'Basal cell carcinoma.9', 'Basal cell carcinoma.10', 'Basal cell carcinoma.11', 'Basal cell carcinoma.12', 'Basal cell carcinoma.13', 'Basal cell carcinoma.14', 'Metastatic Melanoma.8', 'Metastatic Melanoma.9', 'Metastatic Melanoma.10', 'Metastatic Melanoma.11', 'Metastatic

In [19]:
xls = pd.ExcelFile('C:\\Users\\DELL\\Downloads\\GSE7553.XLSX')
print(xls.sheet_names)

['Sheet1']


In [13]:
print(df_raw.columns[:5])


Index(['Basal cell carcinoma', 'Basal cell carcinoma.1',
       'Basal cell carcinoma.2', 'Basal cell carcinoma.3', 'Melanoma in situ'],
      dtype='object')


In [14]:
import os
melanoma_dir = os.path.join(train_dir, "melanoma")
print(os.listdir(melanoma_dir)[:5])


['ISIC_0000139.jpg', 'ISIC_0000141.jpg', 'ISIC_0000142.jpg', 'ISIC_0000143.jpg', 'ISIC_0000144.jpg']


In [52]:
print("Class names from folder structure:", class_names)


Class names from folder structure: ['actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']
